In [110]:
%%html
<style>.container{width:99%}</style>

In [71]:
from pathlib import Path

import numpy as np
import pandas as pd

from reproml.config import etl_conf as ec, ml_conf
from reproml.etl.utils import download_url

In [20]:
df = pd.read_csv(dstpath/Path(url).name)

In [6]:
etl_conf.lyrics.source[0].url

'https://github.com/tmthyjames/cypher/raw/master/notebooks/lyrics1.csv.zip'

In [17]:
dstpath: str = es.root_path / es.raw_data_path / es.lyrics.path
dstpath = Path(dstpath)

for source in es.lyrics.source:
    url, filename = source.get('url'), source.get('filename')
    download_url(url, Path(url).name, dstpath)

In [40]:
lyrics_files = [i for i in dstpath.glob("lyrics*.zip")]

lyrics_df = pd.concat((pd.read_csv(f) for f in lyrics_files), ignore_index=True)

In [57]:
lyrics_df['ranker_genre'] = np.where(
    (lyrics_df['ranker_genre'] == 'screamo')|
    (lyrics_df['ranker_genre'] == 'punk rock')|
    (lyrics_df['ranker_genre'] == 'heavy metal'), 
    'alt rock', 
    lyrics_df['ranker_genre']
)

In [49]:
from sklearn.utils import shuffle
from nltk.corpus import stopwords

In [59]:
group = ['song', 'year', 'album', 'genre', 'artist', 'ranker_genre']
lyrics_by_song = lyrics_df.sort_values(group)\
        .groupby(group).lyric\
        .apply(' '.join)\
        .apply(lambda x: x.lower())\
        .reset_index(name='lyric')

lyrics_by_song["lyric"] = lyrics_by_song['lyric'].str.replace(r'[^\w\s]','')

/var/folders/dv/f_84y86n4bs6c42_c_fpqklh0000gn/T/ipykernel_10271/1233240465.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  lyrics_by_song["lyric"] = lyrics_by_song['lyric'].str.replace(r'[^\w\s]','')


In [61]:
lyrics_by_song['ranker_genre'].value_counts()

alt rock            31472
classic rock        29453
Country             27175
rhythm and blues    13167
Hip Hop             11662
indie folk           6297
pop                  3590
Name: ranker_genre, dtype: int64

In [91]:
pp = ec.root_path / ec.processed_data_path / ec.lyrics.path

lyrics_files = [i for i in pp.glob("*/*")]

lyrics_by_song = pd.concat((pd.read_parquet(f) for f in lyrics_files), ignore_index=True)

In [93]:
msk = np.random.rand(len(lyrics_by_song)) < 0.8

train = lyrics_by_song[msk]

test = lyrics_by_song[~msk]

In [64]:

genres = [
    'Country', 'alt rock', 'Hip Hop',
]

LYRIC_LEN = 400 # each song has to be 400 characters
N = 10000 # number of records to pull from each genre
RANDOM_SEED = 42 # random seed to make results repeatable

train_df = pd.DataFrame()
test_df = pd.DataFrame()
for genre in genres: # loop over each genre
    subset = lyrics_by_song[ # create a subset 
        (lyrics_by_song.ranker_genre==genre) & 
        (lyrics_by_song.lyric.str.len()>=LYRIC_LEN)
    ]
    train_set = subset.sample(n=N, random_state=RANDOM_SEED)
    test_set = subset.drop(train_set.index)
    train_df = train_df.append(train_set) # append subsets to the master sets
    test_df = test_df.append(test_set)
    
train_df = shuffle(train_df)
test_df = shuffle(test_df)

/var/folders/dv/f_84y86n4bs6c42_c_fpqklh0000gn/T/ipykernel_10271/2475517085.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(train_set) # append subsets to the master sets
/var/folders/dv/f_84y86n4bs6c42_c_fpqklh0000gn/T/ipykernel_10271/2475517085.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(test_set)
/var/folders/dv/f_84y86n4bs6c42_c_fpqklh0000gn/T/ipykernel_10271/2475517085.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(train_set) # append subsets to the master sets
/var/folders/dv/f_84y86n4bs6c42_c_fpqklh0000gn/T/ipykernel_10271/2475517085.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [104]:
train

,song,album,genre,artist,ranker_genre,lyric
0,2112,2112 (1976),Hard Rock|Progressive Rock|Art Rock|Psychedeli...,Rush,classic rock,yeah whoa yeah yeah yeah and the meek shall in...
1,A Passage to Bangkok,2112 (1976),Hard Rock|Progressive Rock|Art Rock|Psychedeli...,Rush,classic rock,our first stop is in bogota to check colombian...
3,Something for Nothing,2112 (1976),Hard Rock|Progressive Rock|Art Rock|Psychedeli...,Rush,classic rock,waiting for the winds of change to sweep the c...
5,The Twilight Zone,2112 (1976),Hard Rock|Progressive Rock|Art Rock|Psychedeli...,Rush,classic rock,a pleasant faced man steps up to greet you he ...
6,Always Run To You,7800 Degrees Fahrenheit (1985),Glam Metal|Hard Rock|Country Rock,Bon Jovi,classic rock,the clock strikes ten out on the streets agai...
...,...,...,...,...,...,...
122809,You'll Always Reap What You Sow,Dreamtime (1986),Punk Rock|Rock,The Stranglers,alt rock,to think i knew you when you had nothing at al...
122810,You're All I Need to Get By,30 Greatest Hits (1986),Soul|R&B|Jazz|Blues|Gospel,Aretha Franklin,rhythm and blues,youre all i need to get by youre all i need to...
122811,You're Mine Tonight,Winner In You (1986),R&B|Soul|Funk|Disco|Gospel,Patti LaBelle,rhythm and blues,im the kind of a lady i always get what i want...
122812,You're My Love,They Don't Make Them Like They Used To (1986),Jazz,Kenny Rogers,Country,you wanna know if i care for you you ought to ...


In [106]:
import joblib

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our model
text_clf = Pipeline(
    [('vect', CountVectorizer()),
     ('clf', MultinomialNB(alpha=0.1))])

# train our model on training data
text_clf.fit(train.lyric, train.ranker_genre)  

# score our model on testing data
predicted = text_clf.predict(test.lyric)
np.mean(predicted == test.ranker_genre)

0.5977105478331971

In [211]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

# define our model
text_clf = Pipeline(
    [('vect', TfidfVectorizer()),
     ('clf', MultinomialNB(alpha=0.1))])

# train our model on training data
text_clf.fit(train_df.lyric, train_df.ranker_genre)  

# score our model on testing data
predicted = text_clf.predict(test_df.lyric)
np.mean(predicted == test_df.ranker_genre)

0.8585949256804921

In [115]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [124]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

stop = list(set(stopwords.words('english'))) # stopwords
wnl = WordNetLemmatizer() # lemmatizer

def tokenizer(x): # custom tokenizer
    return (
        wnl.lemmatize(w) 
        for w in word_tokenize(x) 
        if len(w)==2 and w.isalnum() # only words that are  2 characters
    )                                 # and is alpha-numeric

# define our model
text_clf = Pipeline(
    [('vect', TfidfVectorizer(
        ngram_range=(1, 2), # include bigrams
        tokenizer=tokenizer,
        stop_words=stop,
        max_df=0.4, # ignore terms that appear in more than 40% of documents
        min_df=4)), # ignore terms that appear in less than 4 documents
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB(alpha=0.1))])

# train our model on training data
text_clf.fit(train_df.lyric, train_df.ranker_genre)  

# score our model on testing data
predicted = text_clf.predict(test_df.lyric)
np.mean(predicted == test_df.ranker_genre)

/Users/timdobbins/.local/share/virtualenvs/ds-meetup-ml-repro-5j99aBHa/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wo'] not in stop_words.
  warnings.warn(


0.5346252343110595

In [107]:
joblib.dump(text_clf, "../model_artifacts/model.pkl")

['../model_artifacts/model.pkl']

In [229]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

mat = confusion_matrix(test_df.ranker_genre, predicted)
precision = precision_score(test_df.ranker_genre, predicted, average="weighted")
recall = recall_score(test_df.ranker_genre, predicted, average="weighted")
f1 = f1_score(test_df.ranker_genre, predicted, average="weighted")
tps = np.trace(mat)
fps = (np.sum(mat, axis=0) - np.diagonal(mat)).sum()
fns = (np.sum(mat, axis=1) - np.diagonal(mat)).sum()

In [236]:
cm_df

,p_Country,p_alt rock,p_Hip Hop
t_Country,15248,243,859
t_alt rock,32,1322,55
t_Hip Hop,2992,1175,15951


In [226]:
np.diagonal(mat)

array([15248,  1322, 15951])

In [227]:
(np.sum(mat, axis=1) - np.diagonal(mat))

array([1102,   87, 4167])

In [228]:
(np.sum(mat, axis=0) - np.diagonal(mat))

array([3024, 1418,  914])

In [230]:
metrics = {
    "precision": precision.item(),
    "recall": recall.item(),
    "f1": f1.item(),
    "tps": tps.item(), 
    "fns": fns.item(),
    "fps": fps.item()
}

In [231]:
metrics

{'precision': 0.8805240519243002,
 'recall': 0.8585949256804921,
 'f1': 0.8620922064138449,
 'tps': 32521,
 'fns': 5356,
 'fps': 5356}

In [220]:
cm_df = pd.DataFrame(mat, index=['t_' + col for col in genres], columns=['p_' + col for col in genres])

cm_df

,p_Country,p_alt rock,p_Hip Hop
t_Country,15248,243,859
t_alt rock,32,1322,55
t_Hip Hop,2992,1175,15951


In [196]:
cm_df.loc['Country',['Country']], cm_df.loc['Country',['alt rock', 'Hip Hop']]

(Country    10521
 Name: Country, dtype: int64,
 alt rock    1148
 Hip Hop     4681
 Name: Country, dtype: int64)